In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.softmax(x, dim=1)
        return output
    
model = Net()
inp = torch.rand(1, 1, 28, 28)
print(model(inp).shape)

In [ ]:
train_transforms = transforms.Compose([
    # Add random transformations to the image.
    transforms.RandomAffine(
        degrees=30, translate=(0.5, 0.5), scale=(0.25, 1),
        shear=(-30, 30, -30, 30)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  
])

In [ ]:
train_data = torchvision.datasets.MNIST('./data', train=True, download=True,
                                        transform=train_transforms)
test_data = torchvision.datasets.MNIST('./data', train=False, download=True,
                                       transform=test_transforms)

train_loader = torch.utils.data.DataLoader(train_data)
test_loader = torch.utils.data.DataLoader(test_data)

In [ ]:
from torch.optim import Adadelta
from torch.optim.lr_scheduler import StepLR

optimizer = Adadelta(model.parameters(), lr=1.0)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

In [ ]:
import copy
import os
import pickle
from typing import Optional
from tqdm.notebook import tqdm

def train_fn(model: nn.Module,
             data: torch.utils.data.DataLoader,
             epoch: int,
             loss_fn: torch.nn.modules.loss,
             optimizer: torch.optim.Optimizer,
             scheduler: Optional[torch.optim.lr_scheduler.LRScheduler] = None,
             savepath: str = None,
             device='cpu'):

    best_weight = copy.deepcopy(model.state_dict())
    best_loss = float('inf')
    best_acc = 0.0

    if savepath and os.path.exists(savepath):
        # Try loading the model and weight
        try:
            with open(savepath, 'rb') as filehandler:
                prev_train = pickle.load(filehandler)
                best_weight = prev_train['best_weight']
                model.load_state_dict(best_weight)
                best_loss = prev_train['best_loss']
                print(f"Loaded model with loss: {best_loss:0.4f}")
        except:
            print(f"Could not load from path: {savepath}")

    # Gathering all dataset phases
    train_phases = list(data.keys())
    dataset_sizes = {train_phase : len(data[train_phase].dataset) \
                     for train_phase in train_phases}

    for e in range(epoch):
        for phase in train_phases:
            is_training = (phase == 'train')
            model.train() if is_training else model.eval()

            loss, corrects, dats = 0., 0., 0.
            tqdm_data = tqdm(data[phase])

            for x, y in tqdm_data:
                x, y = x.to(device), y.to(device)

                # Clearing optimizer
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    pred = model(x)
                    max_logits, classes = torch.max(pred, 1)
                    batch_loss = loss_fn(pred, y)

                    if phase == 'train':
                        batch_loss.backward()
                        optimizer.step()
                        if scheduler: scheduler.step()

                # Stats
                dats += x.size(0)
                loss += batch_loss.item() * x.size(0)
                corrects += torch.sum(classes == y.data)
                tqdm_data.set_description(f"Epoch {e+1}:: Loss: {loss/dats:.4f}, AC: {corrects/dats:.4f}")

            epoch_loss = loss / dataset_sizes[phase]
            epoch_acc = corrects.double() / dataset_sizes[phase]

            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_weight = copy.deepcopy(model.state_dict())
                if savepath:
                    with open(savepath, 'wb') as filehandler:
                        pickle.dump({
                            'best_weight': best_weight,
                            'best_loss': epoch_loss,
                            'best_acc': best_acc,
                        }, filehandler)
                print(f"Best acc found: {best_acc:3.4f}")

    print(f'Best accuracy: {best_acc:3.4f}')
    model.load_state_dict(best_weight)

In [ ]:
train_fn(
    model = model,
    data = {
        "train": train_loader,
        "test": test_loader,
    },
    epoch = 50,
    loss_fn = F.cross_entropy,
    optimizer = optimizer,
    scheduler = scheduler,
    savepath = "mnist.pkl",
    device = 'cpu'
)